In [1]:
from radd import build, vis
from radd.toolbox import analyze
from __future__ import division

In [5]:
iv = {'v':array([ 1.41106,  1.56252,  1.68275,  1.76071,  1.87995,  1.92475]), 'a': 0.487226, 'xb':   1.563474, 'tr':   0.292126}
ivtr = {'a': 0.06720971, 'xb': 1.467605,  'v':array([ 0.83108,  0.97048,  0.96678,  0.97912,  0.93224,  1.07883]), 'tr':array([ 0.51509,  0.50619,  0.49248,  0.47988,  0.45145,  0.46361])}
itr = {'a': 0.62868, 'v': 1.421123, 'xb': 0.640829, 'tr':array([ 0.18203,  0.16131,  0.13355,  0.11741,  0.08473,  0.07697])}
ia = {'a':array([ 0.37922,  0.34356,  0.30503,  0.28076,  0.24567,  0.23642]), 'xb': 0.9136456, 'v': 1.127346, 'tr': 0.2997148}
ixb={'a': 0.48158, 'xb': array([ 0.04638,  0.1547 ,  0.37575,  0.53875,  1.29523,  1.29254]), 'tr':0.27438, 'v': 1.54521}


names = ['v', 'tr', 'a', 'xb']
inits_list = [iv, itr, ia, ixb]
for i, idc in enumerate(inits_list):
      idc[names[i]] = np.mean(idc[names[i]])

ivtr['v']=np.mean(ivtr['v'])
ivtr['tr']=np.mean(ivtr['tr'])
inits_list.append(ivtr)

In [6]:
parameter = 'xb'
depends_on = {parameter:'pGo'}
finfo_list, yhat_list = [], []
d = '/'.join(['FinalPro', parameter])
if not os.path.isdir(pth+d):
      os.mkdir(pth+d)
os.chdir(pth+d)
finfo_list, yhat_list = [], []

In [ ]:
for i in range(20):
      xinits_list = [dict(deepcopy(xinit)) for xinit in inits_list]
      for xi in xinits_list:
            model = build.Model(data=prodata, kind='xpro', inits=xi, depends_on=depends_on)
            model.make_optimizer(tol=1.e-5, multiopt=True, maxfev=500)
            opt=model.opt
            opt.make_simulator()
            
            xi = model.__nudge_params__(xi)
            ydata, ywts = model.avg_y, model.avg_wts
            # STAGE 3 (Final Simplex)
            yhat, finfo, xiv = opt.gradient_descent(y=ydata, wts=ywts, inits=xi, is_flat=False)
            finfo_list.append(finfo)
            yhat_list.append(yhat)

In [ ]:
dfx = pd.DataFrame([fi.T for fi in finfo_list])
dff = dfx[['cnvrg', 'logp', 'chi', 'rchi', 'AIC', 'BIC', 'a', 'v', 'xb_0', 'xb_20', 'xb_40', 'xb_60', 'xb_80',  'xb_100', 'tr']]
dff.to_csv('xpro_xb_bootinfo.csv', index=False)
yhatdf = pd.DataFrame(np.asarray(yhat_list))
yhatdf.to_csv('xpro_xb_bootfits.csv', index=False)